# Document an Credit Risk Model

Build and document an *credit risk model* with the ValidMind Developer Framework by using Kaggle's [German Credit Risk](https://www.kaggle.com/datasets/uciml/german-credit/data) sample dataset to build a simple credit risk model.

The original dataset contains 1000 entries with 20 categorial/symbolic attributes prepared by Prof. Hofmann. In this dataset, each entry represents a person who takes a credit by a bank. Each person is classified as good or bad credit risks according to the set of attributes. The link to the original dataset can be found below.

This interactive notebook provides a step-by-step guide for loading a demo dataset, preprocessing the raw data, training a model for testing, setting up test inputs, initializing the required ValidMind objects, running the test, and then logging the results to ValidMind.

## Contents    
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Initialize the client library](#toc3_)    
- [Initialize the Python environment](#toc4_)    
  - [Preview the documentation template](#toc4_1_)    
- [Load the sample dataset](#toc5_)    
  - [Prepocess the dataset](#toc5_1_)    
  - [Feature engineering](#toc5_2_)    
- [Train the model](#toc6_)    
  - [Compute probabilities](#toc6_1_)    
  - [Compute binary predictions](#toc6_2_)    
  - [Compute scores](#toc6_3_)    
- [Document the model](#toc7_)    
  - [Initialize the ValidMind datasets](#toc7_1_)    
  - [Initialize a model object](#toc7_2_)    
  - [Assign prediction values and probabilities to the datasets](#toc7_3_)    
  - [Assign scores to the datasets](#toc7_4_)    
  - [Data validation](#toc7_5_)    
    - [Run tests for raw data tests](#toc7_5_1_)    
    - [Run tests for preprocessed data](#toc7_5_2_)    
    - [Run tests for WoE analysis](#toc7_5_3_)    
  - [Model validation](#toc7_6_)    
    - [Run tests for model training](#toc7_6_1_)    
    - [Run tests for model evaluation](#toc7_6_2_)    
    - [Run tests for model explainability](#toc7_6_3_)    
- [Next steps](#toc8_)    
  - [Work with your model documentation](#toc8_1_)    
  - [Discover more learning resources](#toc8_2_)   

<a id='toc1_'></a>

## About ValidMind
ValidMind is a platform for managing model risk, including risk associated with AI and statistical models.

You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin
This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language.

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?
If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/developer/get-started-developer-framework.html), we recommend you begin by exploring the available resources in this section. There, you can learn more about documenting models, find code samples, or read our developer reference.


### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Custom tests**: Custom tests are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

- **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
- **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
- **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom test.
- **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom test. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a test, customize its behavior, or provide additional context.

**Outputs**: Custom tests can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: The [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.

<a id='toc2_'></a>

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
#!pip uninstall -y validmind
!pip install -q ../../../../developer-framework
#!pip -q install aequitas fairgbm vl-convert-python

<a id='toc3_'></a>

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Credit Risk Scorecard`
   - Use case: `Risk Management/CECL`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "967e977f0ef42992391da80009087d44",
  api_secret = "1d56522ecfd230e0c265b6ab1c21ed56d552a9f1c9fc395260500f4e2d4c0b34",
  project = "cm0qnc2yv09ta21ihs1tm8ktl"
)
print(f"ValidMind version: {vm.__version__}")

<a id='toc4_'></a>

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
import pandas as pd
import numpy as np 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector

from validmind.tests import run_test

%matplotlib inline

<a id='toc4_1_'></a>

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You'll upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

<a id='toc5_'></a>

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you'll need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
from validmind.datasets.credit_risk import lending_club_bias as demo_dataset

df = demo_dataset.load_data()

df.head()

In [ ]:
# Save the original labels for the protected classes for visualizations and investigation of biased outcomes
protected_classes_df = df[demo_dataset.protected_classes]

<a id='toc5_1_'></a>

### Prepocess the dataset

In the preprocessing step we perform a number of operations to get ready for building our credit decision model. 

We will in this example, create new feature, fill missing values and encode categorical variables.

In [ ]:
preprocess_df = demo_dataset.preprocess(df)
preprocess_df.head()

<a id='toc6_'></a>

## Train the model

In this section, we focus on constructing and refining our predictive model. 
- We begin by dividing our data into training and testing sets (`train_df`, `test_df`). 
- We employ a simple random split, randomly allocating data points to each set to ensure a mix of examples in both.

In [ ]:
# Split the data into training and testing sets
train_df, test_df = demo_dataset.split(preprocess_df)

X_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
X_test = test_df.drop(demo_dataset.target_column, axis=1)
y_test = test_df[demo_dataset.target_column]

In [ ]:
# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Print feature importances 
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importances:")
print(feature_importances)

# Print model parameters
print("\nModel Parameters:")
print(model.get_params())

# Print basic model information
print(f"\nNumber of trees: {model.n_estimators}")

### Compute probabilities

In [ ]:
train_probabilities = model.predict_proba(X_train)[:,1]
test_probabilities = model.predict_proba(X_test)[:,1]

### Compute binary predictions

In [ ]:
cut_off_threshold = 0.5
train_binary_predictions = (train_probabilities > cut_off_threshold).astype(int)
test_binary_predictions = (test_probabilities > cut_off_threshold).astype(int)

### Compute scores

In this phase, we translate model predictions into actionable scores using probability estimates generated by our trained model. 

In [ ]:
train_scores = demo_dataset.compute_scores(train_probabilities)
test_scores = demo_dataset.compute_scores(test_probabilities)

## Postprocess the dataset

In [ ]:
train_df = train_df.merge(
    protected_classes_df,
    left_index=True,
    right_index=True,
)

test_df = test_df.merge(
    protected_classes_df,
    left_index=True,
    right_index=True,
)

In [ ]:
train_df.head()

<a id='toc7_'></a>

## Document the model

To document the model with the ValidMind Developer Framework, you'll need to:
1. Preprocess the raw dataset
2. Initialize some training and test datasets
3. Initialize a model object you can use for testing
4. Run the full suite of tests

<a id='toc7_1_'></a>

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset`: The dataset that you want to provide as input to tests.
- `input_id`: A unique identifier that allows tracking what inputs are used when running each individual test.
- `target_column`: A required argument if tests require access to true values. This is the name of the target column in the dataset.

With all datasets ready, you can now initialize the raw, training and test datasets created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
# Extract feature columns
feature_columns = train_df.drop(
    columns=[demo_dataset.target_column] + demo_dataset.protected_classes
).columns.tolist()
feature_columns

In [ ]:
vm_raw_ds= vm.init_dataset(
    dataset=df,
    input_id="raw_dataset",
    target_column=demo_dataset.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    input_id="test_dataset",
    target_column=demo_dataset.target_column,
    feature_columns=feature_columns
)

In [ ]:
print(vm_train_ds)

<a id='toc7_2_'></a>

### Initialize a model object

You will also need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="random_forest_model",
)

<a id='toc7_3_'></a>

### Assign prediction values and probabilities to the datasets

With our model now trained, we'll move on to assigning both the predictive probabilities coming directly from the model's predictions, and the binary prediction after applying the cutoff threshold described in the previous steps. 
- These tasks are achieved through the use of the `assign_predictions()` method associated with the VM `dataset` object.
- This method links the model's class prediction values and probabilities to our VM train and test datasets.

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
    prediction_values=train_binary_predictions,
    prediction_probabilities=train_probabilities,
)

vm_test_ds.assign_predictions(
    model=vm_model,
    prediction_values=test_binary_predictions,
    prediction_probabilities=test_probabilities,
)

### Assign scores to the datasets

Credit scorecards revolve around scores computed from model predictions rather than raw predictions, so let's compute some scores!

- To make this process auditable and ensure scores are properly integrated with our datasets, we use the `add_extra_column()` method from the VM `dataset` object.
- This approach allows us to append scores directly to our data, maintaining a streamlined and coherent dataset ready for analysis.

In [ ]:
vm_train_ds.add_extra_column("glm_scores", train_scores)
vm_test_ds.add_extra_column("glm_scores", test_scores)

### Run tests

#### Data Description

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.DatasetDescription",
    inputs={
        "dataset": "raw_dataset",
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.DescriptiveStatistics",
    inputs={
        "dataset": "raw_dataset",
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.TabularNumericalHistograms",
    inputs={
        "dataset": "raw_dataset"
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.TargetRateBarPlots",
    inputs={
        "dataset": "raw_dataset"
    },
    params={
        "default_column": demo_dataset.target_column,
        "columns": None,
    },
)
test.log()

#### Data Quality

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "min_percent_threshold": 10
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.Duplicates",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "min_threshold": 1
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.HighCardinality",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "num_threshold": 100,
        "percent_threshold": 0.1,
        "threshold_type": "percent"
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.MissingValues",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "min_threshold": 1,
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.Skewness",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "max_threshold": 1,
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.UniqueRows",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "min_percent_threshold": 1,
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.TooManyZeroValues",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "max_percent_threshold": 0.03,
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.IQROutliersTable",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "threshold": 1.5,
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.IQROutliersBarPlot",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "threshold": 1.5,
        "fig_width": 800,
    }
)
test.log()

#### Correlations

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.PearsonCorrelationMatrix",
    inputs={
        "dataset": "raw_dataset",
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.HighPearsonCorrelation",
    inputs={
        "dataset": "raw_dataset",
    },
    params={
        "max_threshold": 0.3
    }   
)
test.log()

#### Model Training

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.ModelMetadata",
    inputs={
        "model": "random_forest_model",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.data_validation.DatasetSplit",
    inputs={
        "datasets": ["train_dataset", "test_dataset"],
    }, 
)
test.log()

#### Model Validation

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.PopulationStabilityIndex",
    inputs={
        "model": "random_forest_model",
        "datasets": ["train_dataset", "test_dataset"],
    }, 
    params={
        "num_bins": 10,
        "mode": "fixed"
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.ConfusionMatrix",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.ClassifierPerformance:in_sample",
    inputs={
        "model": "random_forest_model",
        "dataset": "train_dataset",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.ClassifierPerformance:out_of_sample",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.PrecisionRecallCurve",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.ROCCurve",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.TrainingTestDegradation",
    inputs={
        "model": "random_forest_model",
        "datasets": ["train_dataset", "test_dataset"],
    }, 
    params={
        "metrics": ["accuracy", "precision", "recall", "f1"],
        "max_threshold": 0.1
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumAccuracy",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
    params={
        "min_threshold": 0.7
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumF1Score",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
    params={
        "min_threshold": 0.7
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.MinimumROCAUCScore",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
    params={
        "min_threshold": 0.5
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.statsmodels.GINITable",
    input_grid={
        "dataset": [vm_train_ds, vm_test_ds],
        "model": [vm_model],
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.statsmodels.PredictionProbabilitiesHistogram",
    input_grid={
        "dataset": [vm_train_ds, vm_test_ds],
        "model": [vm_model],
    },
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.statsmodels.CumulativePredictionProbabilities",
    input_grid={
        "model": [vm_model],
        "dataset": [vm_train_ds, vm_test_ds],
    },
)
test.log()

#### Model Explainability

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.PermutationFeatureImportance",
    inputs={
        "model": "random_forest_model",
        "dataset": "test_dataset",
    }, 
    params={
        "fontsize": None,
        "figure_height": 1000
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
"validmind.model_validation.sklearn.SHAPGlobalImportance",
inputs={
    "model": "random_forest_model",
    "dataset": "train_dataset",
    }, 
    params={
        "kernel_explainer_samples": 10,
        "tree_or_linear_explainer_samples": 200
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.WeakspotsDiagnosis",
    inputs={
        "model": "random_forest_model",
        "datasets": ["train_dataset", "test_dataset"],
    }, 
    params={
        "features_columns": None,
        "thresholds": {
            "accuracy": 0.75,
            "precision": 0.5,
            "recall": 0.5,
            "f1": 0.7
        }
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.OverfitDiagnosis",
    inputs={
        "model": "random_forest_model",
        "datasets": ["train_dataset", "test_dataset"],
    }, 
    params={
        "metric": None,
        "cut_off_threshold": 0.04
    }
)
test.log()

In [ ]:
test = vm.tests.run_test(
    "validmind.model_validation.sklearn.RobustnessDiagnosis",
    inputs={
        "model": "random_forest_model",
        "datasets": ["train_dataset", "test_dataset"],
    }, 
    params={
        "metric": None,
        "scaling_factor_std_dev_list": [0.1, 0.2, 0.3, 0.4, 0.5],
        "performance_decay_threshold": 0.05
    }
)
test.log()

In [ ]:
#with open('/Users/michaelroenning/solutions_architects_repo/notebooks/test_bias.json', 'r') as f:
#    test_config = json.load(f)

In [ ]:
#full_suite = vm.run_documentation_tests(config=test_config)

#### Bias and Fairness

In [ ]:
test = run_test(
    "validmind.data_validation.ProtectedClassesDescription",
    inputs={
        "dataset": "test_dataset"
    }, 
    params={
        'protected_classes': demo_dataset.protected_classes
    })
test.log()

Now we are going to focus our analysis on the fairness metric(s) of interest in this case study: FNR/FPR across different groups. The aequitas plot module exposes the disparities_metrics() plot, which displays both the disparities and the group-wise metric results side by side.

In [ ]:
test = run_test(
    "validmind.data_validation.ProtectedClassesDisparity", 
    inputs={
        "dataset": "test_dataset"
    }, 
    params={
        "protected_classes": demo_dataset.protected_classes,
        "disparity_tolerance": 1.25
    }
)
test.log()

In [ ]:
run_test(
    "validmind.data_validation.ProtectedClassesCombination", 
    inputs={
        "dataset": "test_dataset"
    }, 
    params={
        "protected_classes": demo_dataset.protected_classes
    }
).log()

In [ ]:
# Define a pipeline for numeric features
numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),  # Impute missing values
        ("scaler", StandardScaler()),  # Scale numeric features
    ]
)

# Define a pipeline for categorical features
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),  # Impute missing values with most frequent
        ("ohe", OneHotEncoder(handle_unknown="ignore")),  # One-hot encode categorical features
    ]
)

# Combine numeric and categorical pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),  # Apply numeric transformer to non-categorical columns
        ("cat", categorical_transformer, selector(dtype_include="category")),  # Apply categorical transformer to categorical columns
    ]
)

# Create the full pipeline including preprocessing and classification
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),  # Apply the preprocessor
        (
            "classifier",
            model,  # Use the previously defined model for classification
        ),
    ]
)

In [ ]:
run_test(
    "validmind.data_validation.ProtectedClassesThresholdOptimizer", 
    inputs={
        "dataset": vm_test_ds
    }, 
    params={
        "pipeline":pipeline,
        "protected_classes": demo_dataset.protected_classes,
        "X_train":X_train,
        "y_train":y_train,
    },
).log()

### Monitoring Section

In [ ]:
#Now let's continue with doing some monitoring and adding metrics over time
df_mon = df = pd.read_csv("lending_club_biased.csv").drop(columns=['total_pymnt'])
df_mon.head()

In [ ]:
#Pre-processing step for prediction purposes
df_mon = df_mon.drop(['id','verification_status','purpose'],axis=1) #remove features we do not want to include for modeling purposes
df_mon2 = df_mon.copy(deep=True)

# Encode categorical variables for modeling purposes
label_encoders = {}
for column in ['term', 'grade', 'sub_grade','emp_length','home_ownership','Gender','Race','Marital_Status']:
    le = LabelEncoder()
    df_mon[column] = le.fit_transform(df_mon[column])
    label_encoders[column] = le

# Display the cleaned and encoded dataset
df_mon.head()

In [ ]:
#splitting into 10 different datasets
df_preds = np.array_split(df_mon,10)


In [ ]:
from datetime import datetime, timedelta
from validmind.api_client import log_metric
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

base_date = datetime.now() - timedelta(days=len(df_preds))

for i in range(len(df_preds)):
    recorded_at = base_date + timedelta(days=i)

    y_pred = model.predict_proba(df_preds[i].drop(columns='loan_status'))
    y_pred_label = model.predict(df_preds[i].drop(columns = 'loan_status'))
    
    #Performance Metrics
    auc = roc_auc_score(df_preds[i]['loan_status'],y_pred[:,1])
    f1 = f1_score(df_preds[i]['loan_status'], y_pred_label, average="macro")
    precision = precision_score(df_preds[i]['loan_status'], y_pred_label, average="macro")
    recall = recall_score(df_preds[i]['loan_status'], y_pred_label, average="macro")
    print(auc,f1,precision,recall)

    #Data Drift Metrics

    log_metric(
        key="demo.metrics.AUC",
        value=auc,
        recorded_at=recorded_at.isoformat(),)

    log_metric(
        key="demo.metrics.F1",
        value=f1,
        recorded_at=recorded_at.isoformat(),)

    log_metric(
        key="demo.metrics.Precision",
        value=precision,
        recorded_at=recorded_at.isoformat(),)

    log_metric(
        key="demo.metrics.Recall",
        value=recall,
        recorded_at=recorded_at.isoformat(),)  


